# Saadi vs. Hafez the Shirazians (Naive Bayes)
In this project, we want to create a classifier for poems. This classifier will train with some poems from two iranian poets, Saadi and Hafez. With this training, we create a model for our classification. New instances predicted with this model and assign to a poet. <br>
## Main idea
I choose __word repetition__ in each poet poems as feature. At first, I calculate this dictionary for training set. And with this values, I calculate different values for bayes rule to predict test set. 
Input of our model is __x (poem)__ and our goal is __c (Poet of that poem)__. <br>
In the bayes rule, we have:<br>
<h1><center>$ P(c|x) = {{P(x|c)P(c)} \over {P(x)}}$</center></h1><br>
We want to calculate $P(c|x)$ that is <strong>posterior probability</strong>. This value means probability of a poet(Saadi or Hafez) given a poem. <br>
<strong>Class prior probability</strong> $P(c)$ is ratio of one poet poems count to all poems. It means probability of that poet before any evidence.<br>
<h4><center>$P(c="Saadi") = {"Saadi" poems\over all poems}$</center></h4><br>
<strong>Evidence</strong> is the poems and by our selected feature is words of that poem. <strong>x</strong> is the poems and it devides to $x_1x_2x_3...x_n$ that each $x_i$ is a word. The probability of evidence is predictor prior probability $P(x)$. This probability in denominator is same for Saadi and hafez given this evidence probability calculation; So we can ignore calculating this and compare numerator of bayes rule for this two poets. <br>
<strong>Likelihood</strong> is probabilty of a poem given a poet. This means how much it's possible that poem is for saadi or hafez. This probability calculated by below formula: <br>
<h4><center>$P(x|c) = P(x_1x_2x_3...x_n|c)$</center></h4><br>
In the naive bayes, we make a simplifier assumption that features are conditionally independent given label. So we can write and calculate it like below: <br>
<h4><center>$P(x_1x_2x_3...x_n|c) = P(x_1|c) \times P(x_2|c) \times ... \times P(x_n|C)$</center></h4><br>
Each $P(x_i|c)$ denotes how much possible that word $x_i$ comes in poems by poet <i>c</i>.<br>
<h4><center>$P(x_i|c) = {P(x_i, c)\over {P(c)}}$</center></h4><br>
P(c) had been discussed above. $P(x_i, c)$ calculated like below:<br>
<h5><center>$P(x_i,c) = {Number Of Word x_i Repetition In c Poems\over Number Of All Words Of c Poems}$</center></h5>

## Initialization
In the first step, we open __train_test.csv__ file and read poems data. Then we devide it to two parts. We choose 80% of this poems randomly as training set and other 20% Then we'll parse poems and split them by space and half space and count words in each poet poems. We count total number of poems by Saadi and Hafez. Our __feature__ in this problem is __words (and their repetition count)__. Our __goal__ is to find poet of the poem.


### Open and reading file
We read file with _pandas csv reader_.

In [330]:
import pandas as pd
train_test_file_path = "./Data/train_test.csv"
train_test_data = pd.read_csv(train_test_file_path)
train_test_data.head()

,text,label
0,چون می‌رود این کشتی سرگشته که آخر,hafez
1,که همین بود حد امکانش,saadi
2,ارادتی بنما تا سعادتی ببری,hafez
3,خدا را زین معما پرده بردار,hafez
4,گویی که در برابر چشمم مصوری,saadi


### Reading stopWords
_Stop Words_ are the most repeated words in each language. We have some of this words in persian literature. We prepare a set of stop words and read them in this part. We remove this words from our training set and hence from our model. With this work, we try to remove not very important feature that got important because they have a very high repeat in poems. 

In [373]:
stop_words_path = "./stopWords.csv"
stop_words = pd.read_csv(stop_words_path)["کلمات"].values
stop_words = list(stop_words)
stop_words.append('')
stop_words

['و',
 'در',
 'به',
 'از',
 'كه',
 'مي',
 'اين',
 'است',
 'را',
 'با',
 'هاي',
 'براي',
 'آن',
 'يك',
 'شود',
 'شده',
 'خود',
 'ها',
 'كرد',
 'شد',
 'اي',
 'تا',
 'كند',
 'بر',
 'بود',
 'نيز',
 'وي',
 'هم',
 'كنند',
 'دارد',
 'ما',
 'كرده',
 'يا',
 'اما',
 'بايد',
 'دو',
 'اند',
 'هر',
 'خواهد',
 'او',
 'نمي',
 'بين',
 'پيش',
 'پس',
 'اگر',
 'همه',
 'صورت',
 'يكي',
 'هستند',
 'بي',
 'من',
 'دهد',
 'هزار',
 'نيست',
 'استفاده',
 'داد',
 'داشته',
 'راه',
 'داشت',
 'چه',
 'همچنين',
 'كردند',
 'داده',
 'بوده',
 'دارند',
 'همين',
 'ميليون',
 'سوي',
 'شوند',
 'بيشتر',
 'بسيار',
 'روي',
 'گرفته',
 'هايي',
 'تواند',
 'هيچ',
 'چند',
 'جديد',
 'بيش',
 'شدن',
 'كردن',
 'كنيم',
 'نشان',
 'حتي',
 'اينكه',
 'ولی',
 'توسط',
 'چنين',
 'برخي',
 'نه',
 'ديروز',
 'دوم',
 'بعد',
 'گيرد',
 'شما',
 'گفته',
 'آنان',
 'بار',
 'طور',
 'گرفت',
 'دهند',
 'طي',
 'بودند',
 'ميليارد',
 'بدون',
 'كل',
 'تر  براساس',
 'شدند',
 'ترين',
 'باشند',
 'ندارد',
 'چون',
 'گويد',
 'ديگري',
 'همان',
 'خواهند',
 'قبل',
 'آمده',


### Devide dataset to train set (80%) and test set (20%)
We should devide our set to two parts. _Training set_ for learning model and _Test set_ to calculate error of our model. We choose this way to avoid overfitting model to training set and get very bad result in operation. <br>
We do this division by make a random number betwwen 0 and 1 and create a mask for data.

In [332]:
import numpy as np

mask = np.random.rand(len(train_test_data)) < 0.8
train_set, test_set = train_test_data[mask], train_test_data[~mask]

### Prepare training set statistics for using in probability calculation of our model
In this part, we explore our training set and collect required data for using in our naive bayes rule.<br>
We calculate poem count of each poet that it's in calculating P(c) and each $P{x_i|c}$. We calculate word repetition in each poet poems. This is the main feature in this problem and it's used to calculate all values of bayes rule. __poem_count__ is count of poet poems. __word_repetition__ is count of each word repetiotion in each poet poems.

In [381]:
import re
import operator

poem_count = {}
word_repetition = {}

def increase_poem_count(poet):
    if poet not in poem_count:
        poem_count[poet] = 1
    else:
        poem_count[poet] += 1

def update_word_repetition(poem, poet):
    if poet not in word_repetition:
        word_repetition[poet] = {}
    words = re.split(' |\u200c', poem)
    for word in words:
        if word not in word_repetition[poet]:
            word_repetition[poet][word] = 1
        else:
            word_repetition[poet][word] += 1

def extract_data(row):
    poet = row['label']
    poem = row['text']
    increase_poem_count(poet)
    update_word_repetition(poem, poet)

train_set.apply(extract_data, axis=1)
print(poem_count)
# print(word_repetition)
# print(sorted(word_repetition["saadi"].items(), key=operator.itemgetter(1)))
# print(sorted(word_repetition["hafez"].items(), key=operator.itemgetter(1)))

{'hafez': 6740, 'saadi': 9944}


### Implement base calculations for naive bayes
This part function are base functions for calculate bayes rule values.<br>
__get_poet_probability__ function calculates P(c) that c is the poet. <br>
__get_partial_likelihood__ function calculates $P{x_i|c}$ that $x_i$ is the word and x is the poet. <br>
__calculate_poets_posterior_probabilty__ function calculates final comparable values for each poet. It's calculate P(c|x) some how because it doesn't calculate P(x) in denominator but it calculates probabilty measure that can compare between two poets. <br> 
It gets a poem and calculate the posterior probability for Hafez and Saadi. It returns this two values. <br>
This function has another input <i>ignore_zeros</i>. If this argument is True, it's ignore words that make probability zero for each poet. If a word have zero partial probability, we ignore it. If different words make two poets probability zero, we calculate the posterior probability another time without this word.

In [308]:
def get_poet_probability(poet):
    return poem_count[poet] / sum(poem_count.values())

def get_partial_likelihood(word, poet):
    poet_probabilty = get_poet_probability(poet)
    poet_words_count = sum(word_repetition[poet].values())
    if word in word_repetition[poet]:
        word_repeat = word_repetition[poet][word]
    else:
        word_repeat = 0
    word_by_poet_probability = word_repeat / poet_words_count
    return word_by_poet_probability / poet_probabilty

def calculate_poets_posterior_probabilty(poem, ignore_zeros = False):
    saadi_prior_probability = get_poet_probability('saadi')
    hafez_prior_probability = get_poet_probability('hafez')
    hafez_likelihood = 1
    saadi_likelihood = 1
    words = re.split(' |\u200c', poem)
    for word in words:
        if word in stop_words:
            continue
        saadi_partial_likelihood = get_partial_likelihood(word, 'saadi')
        hafez_partial_likelihood = get_partial_likelihood(word, 'hafez')
        if ignore_zeros and (saadi_partial_likelihood == 0 or hafez_partial_likelihood == 0):
            continue
        if saadi_partial_likelihood == 0 and hafez_partial_likelihood == 0:
            continue
        saadi_likelihood *= saadi_partial_likelihood
        hafez_likelihood *= hafez_partial_likelihood
    saadi_postrior_probability = saadi_likelihood * saadi_prior_probability
    hafez_postrior_probability = hafez_likelihood * hafez_prior_probability
    return saadi_postrior_probability, hafez_postrior_probability

### Implement predict function
This function get a poem and calculates posterior probability for each poet. By comparing poets, We return the poet name as it's class. <br>
If they have equal probability, We choose class randomly with the chance of it's weight. (Weights are poet poems count.)

In [309]:
def predict_poet(poem):
    saadi_probability, hafez_probability = calculate_poets_posterior_probabilty(poem)
    if saadi_probability == 0 and hafez_probability == 0:
        saadi_probability, hafez_probability = calculate_poets_posterior_probabilty(poem, ignore_zeros=True)
    if saadi_probability > hafez_probability:
        return "saadi"
    elif saadi_probability < hafez_probability:
        return "hafez"
    else:
        random_index = np.random.randint(0, len(train_set)-1)
        if random_index < poem_count["saadi"]:
            return "saadi"
        else:
            return "hafez"

### Evaluation Functions
We have three function for three evaluation function:
1. __calculate_recall__ function calculates below formula:
<h3><center>$Recall = {CorrectDetectedHafezes\over AllHafezes}$</center></h3><br>
2. __calculate_precision__ function calculates below formula:
<h3><center>$Precision = {CorrectDetectedHafezes\over DetectedHafezes}$</center></h3><br>
3. __calculate_accuracy__ function calculates below formula:
<h3><center>$Accuracy = {CorrectDetected\over Total}$</center></h3><br>

In [310]:
def calculate_recall(predicted_labels, actual_labels):
    correct_predicted_hafez = 0
    all_hafez_poems = 0
    for i in range(len(actual_labels)):
        if actual_labels[i] == "hafez":
            all_hafez_poems += 1
            if predicted_labels[i] == "hafez":
                correct_predicted_hafez += 1
    return correct_predicted_hafez / all_hafez_poems

def calculate_precision(predicted_labels, actual_labels):
    correct_predicted_hafez = 0
    all_predicted_hafez = 0
    for i in range(len(actual_labels)):
        if predicted_labels[i] == "hafez":
            all_predicted_hafez += 1
            if actual_labels[i] == "hafez":
                correct_predicted_hafez += 1
    return correct_predicted_hafez / all_predicted_hafez

def calculate_accuracy(predicted_labels, actual_labels):
    all_poems = len(actual_labels)
    correct_predicted_poems = sum([actual_labels[i] == predicted_labels[i] for i in range(len(actual_labels))])
    return correct_predicted_poems / all_poems


### Predict test set
In this part, we predict test set labels and then find three values for errors. The error values are: <br>
1. Recall = 82.03
2. Precision = 62.74
3. Accuracy = 72.99

In [311]:
test_poems = test_set.drop(columns=["label"])
predicted_poets = test_poems["text"].apply(predict_poet)
actual_poets = test_set.drop(columns=["text"])
print(calculate_recall(predicted_poets.values, actual_poets["label"].values))
print(calculate_precision(predicted_poets.values, actual_poets["label"].values))
print(calculate_accuracy(predicted_poets.values, actual_poets["label"].values))

0.8203309692671394
0.6274864376130199
0.7299497246827867


### Laplace smoothing
Assume one word comes in a new poem. This word appear in previopus Saadi's poems but it doesn't appear in Hafez's poems. When we get partial_likelihood for this word and hafez we get 0. When we product it to previous probability, final probability of hafez becoms zero. Therefor it can't be hafez's poem and other words aren't important however they have very strong evidence for being hafez's poem. So our model decide it's saadi's poem and it can be wrong. <br>
The idea for resolve this problem is __Laplace smoothing__. In this situtation, we choose a small probability for the word with 0 repetition. This probability doesn't not remove other evidences effect and doesn't have large impact on choosing a word(decrease probability ver much!).
I have two ideas:
1. Count words with zero repetition in poet previous poems and sum numerator with one and denominator with number of zero-repeated words. Sum of probability in this idea is 1 and we relate a non zero probability to a word with zero repetition.
2. Sum numerator with one and denominator with number of all word counts.<br>
I implementd second idea in __get_partial_laplace_likelihood__.

In [384]:
all_words_count = sum(word_repetition["saadi"].values()) + sum(word_repetition["hafez"].values())

def get_partial_laplace_likelihood(word, poet):
    poet_probabilty = get_poet_probability(poet)
    poet_words_count = sum(word_repetition[poet].values())
    if word in word_repetition[poet]:
        word_repeat = word_repetition[poet][word]
    else:
        word_repeat = 0
    word_by_poet_probability = (word_repeat + 1) / (poem_count[poet] + all_words_count)
    return word_by_poet_probability / poet_probabilty

def calculate_poets_posterior_probabilty_laplace(poem):
    saadi_prior_probability = get_poet_probability('saadi')
    hafez_prior_probability = get_poet_probability('hafez')
    hafez_likelihood = 1
    saadi_likelihood = 1
    words = re.split(' |\u200c', poem)
    for word in words:
        if word not in word_repetition["saadi"] and word not in word_repetition["hafez"]:
            continue
        saadi_partial_likelihood = get_partial_laplace_likelihood(word, 'saadi')
        hafez_partial_likelihood = get_partial_laplace_likelihood(word, 'hafez')
        saadi_likelihood *= saadi_partial_likelihood
        hafez_likelihood *= hafez_partial_likelihood
    saadi_postrior_probability = saadi_likelihood * saadi_prior_probability
    hafez_postrior_probability = hafez_likelihood * hafez_prior_probability
    return saadi_postrior_probability, hafez_postrior_probability

def predict_poet_laplace(poem):
    saadi_probability, hafez_probability = calculate_poets_posterior_probabilty_laplace(poem)
    if saadi_probability > hafez_probability:
        return "saadi"
    elif saadi_probability < hafez_probability:
        return "hafez"
    else:
        random_index = np.random.randint(0, len(train_set)-1)
        if random_index < poem_count["saadi"]:
            return "saadi"
        else:
            return "hafez"

### Test laplace smoothing
Predict test set labels and calculate error values has this results:
1. Recall = 80.39
2. Precision = 70.33
3. Accuracy = 78.64
We see that all evaluation functions have good values and satisfy our needs. We have fair recall and precision and good accuracy.

In [385]:
test_poems = test_set.drop(columns=["label"])
predicted_poets = test_poems["text"].apply(predict_poet_laplace)
actual_poets = test_set.drop(columns=["text"])
print(calculate_recall(predicted_poets.values, actual_poets["label"].values))
print(calculate_precision(predicted_poets.values, actual_poets["label"].values))
print(calculate_accuracy(predicted_poets.values, actual_poets["label"].values))

0.8039332538736591
0.7033368091762252
0.7864447086801427


## Questions:
1. why machine learning model with just high precision is not enough?<br>
This model is not good always. For this problem, Assume a model that predict hafez when it's has ver higher probability than saadi. (For example, 100x) This model is very conservative and just predict hafez in very confident situations. This model has a very high precision(almost 100%) but has a very low recall because it doesn't classify other poems of hafez corrdctly and classify all of them as saadi's poem.
2. Why just good accuracy is not enough for good model?<br>
Assume our model just predict one class. (It set same label for all instances in test set.) Assume we have test dataset that is biased and more than 75% of data belongs to that class. Our not good model with no calculation get more than 75% accuracy in this dataset but it gets very low value of presicion and recall of other class.

### Predict evaluate.csv
Run model against evaluate.csv for final evaluation.

In [396]:
evaluate_file_path = "./Data/evaluate.csv"
evaluate_data = pd.read_csv(evaluate_file_path)
evaluate_poems = evaluate_data.drop(columns=["id"])
predicted_poets = evaluate_poems["text"].apply(predict_poet_laplace)
for i in range(len(predicted_poets)):
    print(predicted_poets[i])
# output_file_path = "./Data/output.csv"
# pd.write_csv(output_file_path) !Lack of net :((

saadi
hafez
saadi
saadi
saadi
saadi
saadi
saadi
saadi
hafez
saadi
hafez
hafez
hafez
saadi
hafez
saadi
saadi
saadi
hafez
saadi
hafez
saadi
saadi
hafez
hafez
hafez
saadi
hafez
saadi
saadi
saadi
hafez
saadi
hafez
hafez
hafez
hafez
saadi
saadi
saadi
hafez
saadi
hafez
saadi
hafez
hafez
hafez
hafez
hafez
hafez
hafez
hafez
hafez
hafez
saadi
hafez
saadi
hafez
saadi
hafez
hafez
hafez
saadi
hafez
hafez
hafez
hafez
saadi
hafez
saadi
saadi
hafez
hafez
hafez
hafez
saadi
hafez
saadi
hafez
hafez
saadi
saadi
hafez
saadi
hafez
saadi
hafez
saadi
hafez
hafez
hafez
saadi
hafez
saadi
saadi
hafez
saadi
hafez
saadi
saadi
hafez
hafez
hafez
saadi
hafez
hafez
hafez
saadi
saadi
saadi
hafez
hafez
saadi
hafez
hafez
saadi
hafez
hafez
hafez
hafez
hafez
saadi
saadi
saadi
saadi
saadi
saadi
saadi
saadi
hafez
saadi
saadi
hafez
saadi
hafez
saadi
saadi
saadi
saadi
saadi
saadi
hafez
hafez
hafez
hafez
hafez
hafez
saadi
saadi
hafez
saadi
hafez
saadi
hafez
saadi
hafez
saadi
hafez
saadi
saadi
hafez
hafez
hafez
hafez
saadi
hafe